In [1]:
#v2 Instead of sorting the ctrl into awake/sleep groups, we just measure the % of flies that are aroused (within # mins described by the variable RISEN) in the ctrl ZT time

import scipy
from scipy import signal, misc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime
import os
import time
import re
import seaborn

from matplotlib import rcParams
plt.rcParams["figure.figsize"] = (20,10)
rcParams['lines.linewidth'] = 3.5
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Tahoma']
font = {'weight' : 'regular',
        'size'   : 30}

matplotlib.rc('font', **font)
plt.style.use('seaborn-colorblind')

def Clean_names(group):
    # Search for opening bracket in the name followed by pattern that looks like CtM01C01$
    if re.search('CtM(\d+)C(\d+)$', group):
          # Extract the position of beginning of pattern
        pos = re.search('CtM(\d+)C(\d+)$', group).start()
        # return the cleaned name
        return group[:pos]  
    else:
        # if no clean up needed return the same name
        return group

In [2]:
#Parameters
inactive=5
risen=3

In [3]:
cwd = os.getcwd()
for root_dir_path, sub_dirs, files in os.walk(cwd):
    for file in files:
        if file.endswith('.txt'):
            print(file)

UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C01.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C07.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C08.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C09.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C10.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C11.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C12.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C13.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C14.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C15.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C16.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C17.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C18.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C19.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C20.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C21.txt
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C22.txt
UASdTRPA1_

In [4]:
#https://stackoverflow.com/questions/26415906/read-multiple-txt-files-into-pandas-dataframe-with-filename-as-column-header
filelist=[]
for root_dir_path, sub_dirs, files in os.walk(cwd):
    for file in files:
        if file.endswith('.txt'):
            filelist.append(file)
sleepdata=pd.concat([pd.read_csv(item, names=[item[:-4]]) for item in filelist], axis=1) #note the -4 part removes the .txt characters that would be in filename

sleepdata

,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C01,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C07,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C08,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C09,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C10,...,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C15,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C16,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C17,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C18,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C19,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C20,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C21,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C22,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C23,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C24
0,MH354R2CtM026C01 31 Jul 2021,MH354R2CtM026C02 31 Jul 2021,MH354R2CtM026C03 31 Jul 2021,MH354R2CtM026C04 31 Jul 2021,MH354R2CtM026C05 31 Jul 2021,MH354R2CtM026C06 31 Jul 2021,MH354R2CtM026C07 31 Jul 2021,MH354R2CtM026C08 31 Jul 2021,MH354R2CtM026C09 31 Jul 2021,MH354R2CtM026C10 31 Jul 2021,...,MH354R2CtM016C15 31 Jul 2021,MH354R2CtM016C16 31 Jul 2021,MH354R2CtM016C17 31 Jul 2021,MH354R2CtM016C18 31 Jul 2021,MH354R2CtM016C19 31 Jul 2021,MH354R2CtM016C20 31 Jul 2021,MH354R2CtM016C21 31 Jul 2021,MH354R2CtM016C22 31 Jul 2021,MH354R2CtM016C23 31 Jul 2021,MH354R2CtM016C24 31 Jul 2021
1,4321,4321,4321,4321,4321,4321,4321,4321,4321,4321,...,4321,4321,4321,4321,4321,4321,4321,4321,4321,4321
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
4,0,0,2,0,2,2,0,2,0,2,...,0,0,0,3,6,5,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4320,0,3,2,2,0,3,3,1,3,1,...,0,0,2,2,1,0,6,0,1,0
4321,0,1,1,2,1,1,2,4,3,1,...,0,0,0,3,1,0,5,0,0,0
4322,0,3,2,4,1,3,2,1,1,2,...,2,1,0,5,3,0,0,0,2,0
4323,0,1,2,3,3,3,2,2,2,0,...,1,2,0,5,0,0,3,0,1,0


In [5]:
sleepdata=sleepdata.iloc[4:] #removes the first four rows of dataframe (metadata)

In [6]:
sleepdata=sleepdata.reset_index(drop=True)
sleepdata #each row is minute, each column is genotype

,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C01,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C07,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C08,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C09,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C10,...,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C15,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C16,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C17,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C18,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C19,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C20,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C21,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C22,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C23,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C24
0,0,0,2,0,2,2,0,2,0,2,...,0,0,0,3,6,5,2,0,0,0
1,0,0,3,0,1,3,2,3,0,1,...,0,0,0,12,6,1,6,0,0,0
2,0,0,5,0,1,3,2,3,0,0,...,0,4,0,2,3,0,2,0,2,0
3,0,0,3,1,0,1,1,0,5,0,...,2,4,0,2,3,4,4,0,11,0
4,0,4,0,2,0,4,1,4,3,3,...,2,4,0,6,3,2,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4316,0,3,2,2,0,3,3,1,3,1,...,0,0,2,2,1,0,6,0,1,0
4317,0,1,1,2,1,1,2,4,3,1,...,0,0,0,3,1,0,5,0,0,0
4318,0,3,2,4,1,3,2,1,1,2,...,2,1,0,5,3,0,0,0,2,0
4319,0,1,2,3,3,3,2,2,2,0,...,1,2,0,5,0,0,3,0,1,0


In [7]:
sleepdata.dtypes

UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C01              object
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02              object
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03              object
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04              object
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05              object
                                                       ...  
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C20    object
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C21    object
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C22    object
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C23    object
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C24    object
Length: 70, dtype: object

In [8]:
for x in sleepdata:
    sleepdata[x]=pd.to_numeric(sleepdata[x])
sleepdata.dtypes

UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C01              int64
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02              int64
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03              int64
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04              int64
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05              int64
                                                      ...  
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C20    int64
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C21    int64
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C22    int64
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C23    int64
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C24    int64
Length: 70, dtype: object

In [9]:
#remove dead animals
#https://stackoverflow.com/questions/21164910/how-do-i-delete-a-column-that-contains-only-zeros-in-pandas
sleepdata = sleepdata.loc[:, (sleepdata != 0).any(axis=0)]
sleepdata

,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C07,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C08,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C09,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C10,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C11,...,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C13,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C14,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C15,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C16,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C17,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C18,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C19,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C20,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C21,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C23
0,0,2,0,2,2,0,2,0,2,2,...,2,6,0,0,0,3,6,5,2,0
1,0,3,0,1,3,2,3,0,1,1,...,2,1,0,0,0,12,6,1,6,0
2,0,5,0,1,3,2,3,0,0,2,...,3,3,0,4,0,2,3,0,2,2
3,0,3,1,0,1,1,0,5,0,3,...,0,1,2,4,0,2,3,4,4,11
4,4,0,2,0,4,1,4,3,3,4,...,3,0,2,4,0,6,3,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4316,3,2,2,0,3,3,1,3,1,0,...,2,1,0,0,2,2,1,0,6,1
4317,1,1,2,1,1,2,4,3,1,0,...,0,1,0,0,0,3,1,0,5,0
4318,3,2,4,1,3,2,1,1,2,0,...,1,4,2,1,0,5,3,0,0,2
4319,1,2,3,3,3,2,2,2,0,0,...,1,1,1,2,0,5,0,0,3,1


In [10]:
#for each genotype, find row that corresponds to stimulus times
sr=1 #sampling rate is 1 minute
shakeday=2
#shakeZTtimes=[14,16,18,20,22] #i.e. ZT14, ZT16, ZT18, ZT20, ZT22
shakeZTtimes=[18] #i.e. ZT14, ZT16, ZT18, ZT20, ZT22
daymins=(shakeday-1)/sr*60*24 #calculates the number of minutes to get to ZT0 of shakeday
shakeZTmins=[int(x/sr*60+daymins) for x in shakeZTtimes] #calculates num of mins at each shakeZT of shakeday
shakeZTmins

[2520]

In [11]:
#sleepdata.reset_index().plot(x='index', y='arousalthresh_iso31_xR19G02GAL4_5xiso_D_MH236R2CtM027C01')

In [12]:
###sleepdata_T = sleepdata.T 
###sleepdata_T.index.name = 'ID'
###sleepdata_T.reset_index(inplace=True)

In [13]:
###sleepdata_T['genotype'] = sleepdata_T['ID'].apply(Clean_names)
###sleepdata_T

In [14]:
#sleepdata.iloc[3270:]
for x in shakeZTmins:
    prestim=sleepdata.iloc[x-inactive:x]
    poststim=sleepdata.iloc[x:x+risen]
    #within_animal control for how much arousal
    y=x-30 #30 min before
    ctrlrisen=sleepdata.iloc[y:y+risen]

In [15]:
prestim
poststim
ctrlrisen

,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C07,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C08,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C09,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C10,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C11,...,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C13,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C14,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C15,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C16,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C17,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C18,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C19,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C20,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C21,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C23
2490,2,2,0,2,7,4,2,4,3,4,...,0,0,0,0,0,0,0,0,0,0
2491,3,4,0,1,5,4,3,4,2,4,...,0,0,0,0,0,0,0,0,0,0
2492,3,4,0,3,7,1,3,1,2,3,...,0,0,0,0,0,0,0,0,0,0


In [16]:
clmn = list(sleepdata) #names of all the columns
#print(clmn)

In [17]:
##genotype names
#https://www.w3schools.com/python/python_howto_remove_duplicates.asp
genotype_list=[]
for x in clmn:
    clmn_short=re.sub(r'CtM(\d+)C(\d+)$', '', x)
    genotype_list.append(clmn_short)
#print(genotype_list)
#genotype_dict = dict.fromkeys(genotype_list)

##number of flies per genotype
#https://www.geeksforgeeks.org/counting-the-frequencies-in-a-list-using-dictionary-in-python/
genotype_dict={}
for items in genotype_list:
        genotype_dict[items] = genotype_list.count(items)
print(genotype_dict)


{'UASdTRPA1_5xiso_x_iso31_MH354R2': 20, 'UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2': 24, 'UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2': 22}


In [18]:
#number of flies asleep
#number of flies awake
#https://stackoverflow.com/questions/18062135/combining-two-series-into-a-dataframe-in-pandas
prestimbeam=pd.Series(prestim.sum(axis=0), name="prestimbeam")
poststimbeam=pd.Series(poststim.sum(axis=0), name="poststimbeam")
ctrlrisen=pd.Series(ctrlrisen.sum(axis=0), name="ctrlrisen")

beamcrosses=pd.concat([prestimbeam, poststimbeam, ctrlrisen], axis=1)
beamcrosses

,prestimbeam,poststimbeam,ctrlrisen
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,11,13,8
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,20,11,10
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,0,12,0
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,16,13,6
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,24,15,19
...,...,...,...
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C18,0,1,0
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C19,0,7,0
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C20,0,4,0
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C21,0,1,0


In [19]:
#type(beamcrosses)
beamcrosses.dtypes

prestimbeam     int64
poststimbeam    int64
ctrlrisen       int64
dtype: object

In [20]:
#https://oindrilasen.com/2019/08/convert-integer-to-boolean-and-vice-versa-in-pandas-dataframe/
#possible states
#state1 = either awake (True) or asleep (False)
#state2 = either awake (True) or asleep (False)
boolstates=beamcrosses.astype('bool') #https://oindrilasen.com/2019/08/convert-integer-to-boolean-and-vice-versa-in-pandas-dataframe/
boolstates


,prestimbeam,poststimbeam,ctrlrisen
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,True,True,True
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,True,True,True
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,False,True,False
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,True,True,True
UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,True,True,True
...,...,...,...
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C18,False,True,False
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C19,False,True,False
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C20,False,True,False
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM016C21,False,True,False


In [21]:
boolstates.index.name = 'ID'
boolstates.reset_index(inplace=True)

In [22]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html
#https://www.geeksforgeeks.org/replace-values-in-pandas-dataframe-using-regex/
#boolstates.reset_index(inplace=True)

boolstates['genotype']=boolstates['ID']
boolstates['genotype'] = boolstates['genotype'].apply(Clean_names)
boolstates_copy=boolstates

#arousal percentage of flies awake
#arousal percentage of flies asleep

In [23]:
#https://stackoverflow.com/questions/34236317/how-to-select-a-list-of-rows-by-name-in-pandas-dataframe
#https://thispointer.com/pandas-6-different-ways-to-iterate-over-rows-in-a-dataframe-update-while-iterating-row-by-row/
###teststate=[]
###ctrlstate=[]

#sleepwake
#sleepsleep
#wakesleep
#wakewake

###for i in range(len(boolstates)) :
###  if(boolstates.iloc[i, 1] ==False and boolstates.iloc[i, 2]==True):
###        teststate.append("sleepwake") #aroused
###  elif(boolstates.iloc[i, 1] ==False and boolstates.iloc[i, 2]==False):
###        teststate.append("sleepsleep") #notaroused
###  elif(boolstates.iloc[i, 1] ==True and boolstates.iloc[i, 2]==False):
###        teststate.append("wakesleep") #ignore
###  elif(boolstates.iloc[i, 1] ==True and boolstates.iloc[i, 2]==True):
###        teststate.append("wakewake") #ignore
#print(teststate)

###for i in range(len(boolstates)) :
###  if(boolstates.iloc[i, 3] ==False and boolstates.iloc[i, 4]==True):
###        ctrlstate.append("sleepwake") #aroused
###  elif(boolstates.iloc[i, 3] ==False and boolstates.iloc[i, 4]==False):
###        ctrlstate.append("sleepsleep") #notaroused
###  elif(boolstates.iloc[i, 3] ==True and boolstates.iloc[i, 4]==False):
###        ctrlstate.append("wakesleep") #ignore
###  elif(boolstates.iloc[i, 3] ==True and boolstates.iloc[i, 4]==True):
###        ctrlstate.append("wakewake") #ignore
#print(ctrlstate)

In [24]:
###boolstates['teststate']=teststate
###boolstates['ctrlstate']=ctrlstate

In [25]:
###boolstates

In [26]:
boolstates_copy

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
0,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
1,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
2,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,False,True,False,UASdTRPA1_5xiso_x_iso31_MH354R2
3,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
4,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
...,...,...,...,...,...
61,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM01...,False,True,False,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2
62,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM01...,False,True,False,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2
63,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM01...,False,True,False,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2
64,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2CtM01...,False,True,False,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2


In [27]:
#https://www.kite.com/blog/python/pandas-groupby-count-value-count/
a = boolstates.groupby('genotype')
print(a.groups)

{'UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43], 'UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2': [44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65], 'UASdTRPA1_5xiso_x_iso31_MH354R2': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}


In [28]:
#type(a)
#a.describe()

In [29]:
expt_bool=boolstates_copy.groupby(['genotype', 'prestimbeam'])
expt_bool.groups

{('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', False): [20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43], ('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', True): [29, 35], ('UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2', False): [45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65], ('UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2', True): [44, 54], ('UASdTRPA1_5xiso_x_iso31_MH354R2', False): [2, 13], ('UASdTRPA1_5xiso_x_iso31_MH354R2', True): [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19]}

In [30]:
ctrl_bool=boolstates_copy.groupby(['genotype', 'ctrlrisen'])
ctrl_bool.groups

{('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', False): [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], ('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', True): [20, 43], ('UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2', False): [45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65], ('UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2', True): [44, 52], ('UASdTRPA1_5xiso_x_iso31_MH354R2', False): [2, 13, 15], ('UASdTRPA1_5xiso_x_iso31_MH354R2', True): [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 19]}

In [31]:
#https://www.kite.com/blog/python/pandas-groupby-count-value-count/
for identifier,group in expt_bool:
    display(identifier)
    display(group)
    display(type(group))

('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', False)

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
20,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,True,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
21,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
22,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
23,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
24,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
25,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,False,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
26,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
27,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
28,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,False,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
30,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2


pandas.core.frame.DataFrame

('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', True)

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
29,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,True,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
35,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,True,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2


...

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
2,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,False,True,False,UASdTRPA1_5xiso_x_iso31_MH354R2
13,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C15,False,True,False,UASdTRPA1_5xiso_x_iso31_MH354R2


pandas.core.frame.DataFrame

('UASdTRPA1_5xiso_x_iso31_MH354R2', True)

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
0,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
1,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
3,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
4,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
5,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C07,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
6,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C08,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
7,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C09,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
8,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C10,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
9,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C11,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
10,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C12,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2


pandas.core.frame.DataFrame

In [32]:
for identifier,group in ctrl_bool:
    display(identifier)
    display(group)
    display(type(group))

('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', False)

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
21,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
22,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
23,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
24,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
25,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,False,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
26,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
27,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
28,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,False,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
29,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,True,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
30,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2


pandas.core.frame.DataFrame

('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', True)

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
20,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,True,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
43,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,False,True,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2


...

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
2,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,False,True,False,UASdTRPA1_5xiso_x_iso31_MH354R2
13,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C15,False,True,False,UASdTRPA1_5xiso_x_iso31_MH354R2
15,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C18,True,True,False,UASdTRPA1_5xiso_x_iso31_MH354R2


pandas.core.frame.DataFrame

('UASdTRPA1_5xiso_x_iso31_MH354R2', True)

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
0,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
1,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
3,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
4,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
5,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C07,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
6,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C08,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
7,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C09,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
8,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C10,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
9,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C11,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
10,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C12,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2


pandas.core.frame.DataFrame

In [33]:
# this gives the number of total sleeping (prestimbeam = False) and total awake (prestimbeam = True)
#boolstates_copy.groupby(["genotype", "prestimbeam"], as_index=False)["poststimbeam"].count()
expt_bool_n=boolstates_copy.groupby(["genotype", "prestimbeam"], as_index=False)["ID"].count()
expt_bool_n

,genotype,prestimbeam,ID
0,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,False,22
1,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2
2,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,False,20
3,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2
4,UASdTRPA1_5xiso_x_iso31_MH354R2,False,2
5,UASdTRPA1_5xiso_x_iso31_MH354R2,True,18


In [34]:
ctrl_bool_n=boolstates_copy.groupby(["genotype", "ctrlrisen"], as_index=False)["ID"].count()
ctrl_bool_n

,genotype,ctrlrisen,ID
0,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,False,22
1,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2
2,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,False,20
3,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2
4,UASdTRPA1_5xiso_x_iso31_MH354R2,False,3
5,UASdTRPA1_5xiso_x_iso31_MH354R2,True,17


In [35]:

ctrl_n=boolstates_copy.groupby(["genotype"])["ID"].count()
ctrl_n

genotype
UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2    24
UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2       22
UASdTRPA1_5xiso_x_iso31_MH354R2                 20
Name: ID, dtype: int64

In [36]:
df1 = pd.DataFrame(columns=["Genotype", "ExptPreStim"])
df2 = pd.DataFrame(columns=["ExptAwakeAfterStim"])
for identifier,group in expt_bool:
    display(identifier)
    display(group)
#    display(type(group))
#    display(identifier[0])
    x = pd.Series([identifier[0],identifier[1]])
    df1 = df1.append({'Genotype': x[0], 'ExptPreStim': x[1]}, ignore_index=True)
    display(group["poststimbeam"].sum()) #this tells you how many poststimbeam in group is "True" (i.e. wake)
    y=group["poststimbeam"].sum()
    df2 = df2.append({'ExptAwakeAfterStim': y}, ignore_index=True)

('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', False)

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
20,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,True,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
21,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
22,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
23,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
24,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
25,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,False,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
26,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
27,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
28,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,False,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
30,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,False,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2


18

('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', True)

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
29,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,True,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2
35,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2Ct...,True,True,False,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2


...

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
2,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C04,False,True,False,UASdTRPA1_5xiso_x_iso31_MH354R2
13,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C15,False,True,False,UASdTRPA1_5xiso_x_iso31_MH354R2


2

('UASdTRPA1_5xiso_x_iso31_MH354R2', True)

,ID,prestimbeam,poststimbeam,ctrlrisen,genotype
0,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C02,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
1,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C03,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
3,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C05,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
4,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C06,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
5,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C07,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
6,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C08,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
7,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C09,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
8,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C10,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
9,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C11,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2
10,UASdTRPA1_5xiso_x_iso31_MH354R2CtM026C12,True,True,True,UASdTRPA1_5xiso_x_iso31_MH354R2


18

In [37]:
df3 = pd.DataFrame(columns=["genotype", "AwakeCtrl"])
for identifier,group in ctrl_bool:
    display(identifier)
#    display(group)
#    display(type(group))
#    display(identifier[0])
    x = pd.Series([identifier[0],identifier[1]])
    df3 = df3.append({'genotype': x[0], 'AwakeCtrl': x[1]}, ignore_index=True)
    display(group["ctrlrisen"].sum()) #this tells you how many postctrlbeam in group is "True" (i.e. wake)

('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', False)

0

('UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2', True)

2

('UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2', False)

...

2

('UASdTRPA1_5xiso_x_iso31_MH354R2', False)

0

('UASdTRPA1_5xiso_x_iso31_MH354R2', True)

17

In [38]:
Expt_df=pd.concat([df1, expt_bool_n["ID"], df2, ], axis=1)
Expt_df

,Genotype,ExptPreStim,ID,ExptAwakeAfterStim
0,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,False,22,18
1,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2,2
2,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,False,20,19
3,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2,2
4,UASdTRPA1_5xiso_x_iso31_MH354R2,False,2,2
5,UASdTRPA1_5xiso_x_iso31_MH354R2,True,18,18


In [39]:
Ctrl_df=pd.concat([df3, ctrl_bool_n["ID"]], axis=1)
Ctrl_df2=pd.merge(Ctrl_df, ctrl_n, on='genotype', how='left')
Ctrl_df2=Ctrl_df2[(Ctrl_df2['AwakeCtrl']==True)]
Ctrl_df2.columns = ['Genotype', "AwakeCtrl", "Number Aroused Ctrl", "Total n Ctrl"]
Ctrl_df2

,Genotype,AwakeCtrl,Number Aroused Ctrl,Total n Ctrl
1,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2,24
3,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2,22
5,UASdTRPA1_5xiso_x_iso31_MH354R2,True,17,20


In [40]:
Expt_df.columns = ['Genotype', "Expt_Initial", "Total n Expt", "Number Aroused Expt"]
Expt_df

,Genotype,Expt_Initial,Total n Expt,Number Aroused Expt
0,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,False,22,18
1,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2,2
2,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,False,20,19
3,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2,2
4,UASdTRPA1_5xiso_x_iso31_MH354R2,False,2,2
5,UASdTRPA1_5xiso_x_iso31_MH354R2,True,18,18


In [41]:
for identifier,group in Expt_df.groupby(["Genotype"])["Expt_Initial"]:
    display(identifier)
    display(group)

'UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2'

0    False
1     True
Name: Expt_Initial, dtype: object

'UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2'

2    False
3     True
Name: Expt_Initial, dtype: object

'UASdTRPA1_5xiso_x_iso31_MH354R2'

4    False
5     True
Name: Expt_Initial, dtype: object

In [42]:
fulldf=pd.merge(Ctrl_df2, Expt_df, on='Genotype', how='inner')
fulldf

,Genotype,AwakeCtrl,Number Aroused Ctrl,Total n Ctrl,Expt_Initial,Total n Expt,Number Aroused Expt
0,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2,24,False,22,18
1,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2,24,True,2,2
2,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2,22,False,20,19
3,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2,22,True,2,2
4,UASdTRPA1_5xiso_x_iso31_MH354R2,True,17,20,False,2,2
5,UASdTRPA1_5xiso_x_iso31_MH354R2,True,17,20,True,18,18


In [43]:
awake_df = fulldf[(fulldf['Expt_Initial']==True)]
display(awake_df)

,Genotype,AwakeCtrl,Number Aroused Ctrl,Total n Ctrl,Expt_Initial,Total n Expt,Number Aroused Expt
1,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2,24,True,2,2
3,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2,22,True,2,2
5,UASdTRPA1_5xiso_x_iso31_MH354R2,True,17,20,True,18,18


In [44]:
asleep_df = fulldf[(fulldf['Expt_Initial']==False)]
display(asleep_df)

,Genotype,AwakeCtrl,Number Aroused Ctrl,Total n Ctrl,Expt_Initial,Total n Expt,Number Aroused Expt
0,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2,24,False,22,18
2,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2,22,False,20,19
4,UASdTRPA1_5xiso_x_iso31_MH354R2,True,17,20,False,2,2


In [45]:
#calculate % aroused by stimulus
asleep_expt_aroused=(asleep_df['Number Aroused Expt']/asleep_df['Total n Expt']*100)
asleep_ctrl_aroused=(asleep_df['Number Aroused Ctrl']/asleep_df['Total n Ctrl']*100)
asleep_df=pd.concat([asleep_df, asleep_expt_aroused, asleep_ctrl_aroused], axis=1)
asleep_df.columns = ["Genotype", "Ctrl_Initial", "Total n Ctrl", "Number Aroused Ctrl", "Expt_Initial", "Total n Expt", "Number Aroused Expt", "% asleep_expt_aroused", "% asleep_ctrl_aroused"]
asleep_df

,Genotype,Ctrl_Initial,Total n Ctrl,Number Aroused Ctrl,Expt_Initial,Total n Expt,Number Aroused Expt,% asleep_expt_aroused,% asleep_ctrl_aroused
0,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2,24,False,22,18,81.818182,8.333333
2,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2,22,False,20,19,95.0,9.090909
4,UASdTRPA1_5xiso_x_iso31_MH354R2,True,17,20,False,2,2,100.0,85.000000


In [46]:
#((awake_df['Number Aroused Expt']/awake_df['Total n Expt'])-(awake_df['Number Aroused Ctrl']/awake_df['Total n Ctrl'])).to_frame('Expt%Aroused-Ctrl%Aroused')
#calculate % aroused by stimulus
awake_expt_aroused=(awake_df['Number Aroused Expt']/awake_df['Total n Expt']*100)
awake_ctrl_aroused=(awake_df['Number Aroused Ctrl']/awake_df['Total n Ctrl']*100)
awake_df=pd.concat([awake_df, awake_expt_aroused, awake_ctrl_aroused], axis=1)
awake_df.columns = ["Genotype", "Ctrl_Initial", "Total n Ctrl", "Number Aroused Ctrl", "Expt_Initial", "Total n Expt", "Number Aroused Expt", "% awake_expt_aroused", "% awake_ctrl_aroused"]
awake_df

,Genotype,Ctrl_Initial,Total n Ctrl,Number Aroused Ctrl,Expt_Initial,Total n Expt,Number Aroused Expt,% awake_expt_aroused,% awake_ctrl_aroused
1,UASdTRPA1_5xiso_x_R19G02GAL4_D_5xiso_MH354R2,True,2,24,True,2,2,100.0,8.333333
3,UASdTRPA1_5xiso_x_SS50574_A_5xiso_MH354R2,True,2,22,True,2,2,100.0,9.090909
5,UASdTRPA1_5xiso_x_iso31_MH354R2,True,17,20,True,18,18,100.0,85.000000
